In [1]:
# import torch
# import numpy as np
# import random
# seed = 45

# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.deterministic = False

import sys
sys.path.append('./models')
from models.BBSNet_model import BBSNetTransformerAttention
from models.BBSNet_model import BBSNetChannelSpatialAttention
from BBSNet_train import setup_gpu, get_model, get_data, main
from BBSNet_test import test
from utils import evaluate, draw_curve

%load_ext autoreload
%autoreload 2

# Train

In [2]:
setup_gpu()
model, optimizer, CE = get_model()
train_loader, test_loader, total_step = get_data()
main(optimizer, model, train_loader, test_loader, CE, total_step)

USE GPU 0
load data...
Start train...
2025-08-26 16:02:25.861429 Epoch [001/002], Step [0001/1092], Loss1: 0.6840 Loss2: 0.7081
2025-08-26 16:02:47.326195 Epoch [001/002], Step [0100/1092], Loss1: 0.2929 Loss2: 0.2676
2025-08-26 16:03:09.036774 Epoch [001/002], Step [0200/1092], Loss1: 0.1693 Loss2: 0.2260
2025-08-26 16:03:30.823782 Epoch [001/002], Step [0300/1092], Loss1: 0.2930 Loss2: 0.3648
2025-08-26 16:03:52.657869 Epoch [001/002], Step [0400/1092], Loss1: 0.7598 Loss2: 0.3225
2025-08-26 16:04:14.746342 Epoch [001/002], Step [0500/1092], Loss1: 0.2336 Loss2: 0.1564
2025-08-26 16:04:36.709315 Epoch [001/002], Step [0600/1092], Loss1: 0.4029 Loss2: 0.2808
2025-08-26 16:04:58.662373 Epoch [001/002], Step [0700/1092], Loss1: 0.3354 Loss2: 0.2683
2025-08-26 16:05:20.598387 Epoch [001/002], Step [0800/1092], Loss1: 0.3810 Loss2: 0.2741
2025-08-26 16:05:42.490124 Epoch [001/002], Step [0900/1092], Loss1: 0.3274 Loss2: 0.1527
2025-08-26 16:06:04.484400 Epoch [001/002], Step [1000/1092], 

# Use Trained Model

I save the model using this code:
torch.save(model.state_dict(), save_path +
                           'BBSNet_epoch_best.pth')

Now, after a long training period, I want to use it on my test data. So, how can I load this model?

In [3]:
from models.BBSNet_model import BBSNetTransformerAttention

model = BBSNetTransformerAttention()

# assume your model is called `model`
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {total_params - trainable_params}")


Total parameters: 58502052
Trainable parameters: 58502052
Non-trainable parameters: 0


In [4]:
# test(method='bbsnet') 